# On and Off Statistics

In [1]:

# Let's first load all required packages...
import os
import pandas as pd
import numpy  as np

from config import *
from bball_stats import *
import tools

In [2]:
# game_id = 2004609  # 2 overtimes
# game_id = 1976446  # different names capitalisation - 6 players
# game_id = 2031329 # players come 
# game_id = 2004608  # Period 2 05.600 - B. Kuol (team 2) come out, but then in and out

# game_id = 1976449   # Makers goes in (for Martin) even though he is already there! So 4 players are left with Martin off
# game_id = 1976457   # same issue with Makers, he goes in even though he is on court!

game_id = 2031329

Process game and extract all data structures (e.g., play-by-plays, stints):

In [3]:
set_logging("INFO")

game_json = tools.get_json_data(game_id)
starters = get_starters(game_json, 1)
pbp_df = get_pbp_df(game_json)
stints = pbp_stints_extract(pbp_df, starters, 1)

result = build_game_stints_stats_df(game_id)
stats_df = result['stint_stats_df']
pbp_df = result['pbp_df']
team1 = result['teams'][0]
team2 = result['teams'][1]
stints1_df = result['stints_df'].query('tno == 1')
stints2_df = result['stints_df'].query('tno == 2')

stints1_df

2022-07-17 15:41:19 WARNING Sub team 1 at 00:07:43: outcoming players not in court: {'H. Besson'}
2022-07-17 15:41:19 WARNING Sub team 1 at 00:01:44: incoming players already in court: {'O. Dieng'}
2022-07-17 15:41:19 WARNING Sub team 1 at 00:07:43: outcoming players not in court: {'H. Besson'}
2022-07-17 15:41:19 WARNING Sub team 1 at 00:01:44: incoming players already in court: {'O. Dieng'}
2022-07-17 15:41:19 WARNING Sub team 2 at 00:10:00: incoming players already in court: {'C. Goulding'}
2022-07-17 15:41:19 WARNING Sub team 2 at 00:05:42: incoming players already in court: {'M. Peatling'}
2022-07-17 15:41:19 WARNING Sub team 2 at 00:05:42: outcoming players not in court: {'S. Ili', 'J. White'}


,id,lineup,intervals,mins,tno,team
0,1,"[F. Delany, H. Besson, O. Dieng, W. McDowell-W...","[(1, 00:10:00, 00:07:15)]",2.750000,1,New Zealand Breakers
1,2,"[F. Delany, J. Martin, O. Dieng, P. Siva, Y. W...","[(1, 00:07:15, 00:03:48), (3, 00:10:00, 00:06:...",7.033333,1,New Zealand Breakers
2,3,"[F. Delany, H. Besson, J. Martin, P. Siva, S. ...","[(1, 00:03:48, 00:02:10)]",1.633333,1,New Zealand Breakers
3,4,"[H. Besson, J. Martin, K. Galloway, P. Siva, S...","[(1, 00:02:10, 00:00:37.800000)]",1.533333,1,New Zealand Breakers
4,5,"[H. Besson, J. Martin, K. Galloway, P. Siva, Y...","[(1, 00:00:37.800000, 00:00:00), (2, 00:10:00,...",0.616667,1,New Zealand Breakers
5,6,"[H. Besson, J. Martin, K. Galloway, W. McDowel...","[(2, 00:10:00, 00:08:34)]",1.433333,1,New Zealand Breakers
6,7,"[F. Delany, H. Besson, J. Martin, W. McDowell-...","[(2, 00:08:34, 00:05:44)]",2.833333,1,New Zealand Breakers
7,8,"[F. Delany, H. Besson, P. Siva, W. McDowell-Wh...","[(2, 00:05:44, 00:01:53)]",3.850000,1,New Zealand Breakers
8,9,"[H. Besson, J. Martin, P. Siva, W. McDowell-Wh...","[(2, 00:01:53, 00:00:00), (3, 00:10:00, 00:10:...",1.883333,1,New Zealand Breakers
9,10,"[F. Delany, J. Martin, O. Dieng, P. Siva, S. T...","[(3, 00:06:25, 00:03:43)]",2.700000,1,New Zealand Breakers


Check if there are plays with no player assigned.

In principle, there should be none, but there are usually some, e.g., in rebounds; guess it's bc they missed the relevant player in the play?

In [4]:
# find plays with no player assigned! Should be empty 
pbp_df.query('player == "nan. nan" and ((tno == 1) or (tno == 2))')

,team_name,team_short_name,clock,s1,s2,lead,tno,period,periodType,pno,player,success,actionType,actionNumber,previousAction,qualifier,subType,scoring,stint1,stint2
483,New Zealand Breakers,Breakers,00:07:29,5,5,0,1,1,REGULAR,0,nan. nan,1,rebound,36,34,[team],offensive,0,1,1
436,Melbourne United,United,00:04:23,8,11,-3,2,1,REGULAR,0,nan. nan,1,rebound,96,94,[team],defensive,0,2,4
423,New Zealand Breakers,Breakers,00:03:29,11,15,-4,1,1,REGULAR,0,nan. nan,1,timeout,115,,[],full,0,3,5
330,New Zealand Breakers,Breakers,00:05:44,33,30,3,1,2,REGULAR,0,nan. nan,1,timeout,682,,[],full,0,7,7
310,Melbourne United,United,00:04:22,34,32,2,2,2,REGULAR,0,nan. nan,1,rebound,266,264,[team],defensive,0,8,8
277,New Zealand Breakers,Breakers,00:01:41,39,34,5,1,2,REGULAR,0,nan. nan,1,turnover,681,,[team],24sec,0,9,9
225,Melbourne United,United,00:07:31,45,45,0,2,3,REGULAR,0,nan. nan,1,rebound,383,380,[team],offensive,0,2,11
189,Melbourne United,United,00:03:43,56,54,2,2,3,REGULAR,0,nan. nan,1,timeout,425,,[],full,0,10,7
151,Melbourne United,United,00:00:55.400000,58,63,-5,2,3,REGULAR,0,nan. nan,1,rebound,463,462,[team],defensive,0,11,14
108,New Zealand Breakers,Breakers,00:08:04,64,71,-7,1,4,REGULAR,0,nan. nan,1,timeout,527,,[],full,0,13,15


In [5]:
# remove NaN players
players_1 = list(pbp_df.query('tno == 1')['player'].unique())
players_2 = list(pbp_df.query('tno == 2')['player'].unique())

players_1.remove('nan. nan')
players_2.remove('nan. nan')

players = players_1
players
# pbp_df.query('tno == 2').loc[players_2.isna()]

['Y. Wetzell',
 'F. Delany',
 'H. Besson',
 'O. Dieng',
 'W. McDowell-White',
 'P. Siva',
 'J. Martin',
 'S. Timmins',
 'K. Galloway',
 'R. Loe']

In [8]:
def get_stints_player(stints_df: pd.DataFrame, player: str):
    mask = stints_df.apply(lambda x: player in x['lineup'], axis=1)

    return stints_df.loc[mask, 'id'].to_list()

# stints1_df
players_stints = dict({})
for x in players:
    players_stints[x] = get_stints_player(stints1_df, x)

players_stints

NameError: name 'get_players_stints' is not defined

In [76]:
# Performance of H. Besson with/without W. McDowell-White
x = 'H. Besson'
y = 'W. McDowell-White'

x_stints = set(players_stints[x])
y_stints = set(players_stints[y])

print(f"Stints where {x} played:", x_stints)
print(f"Stints where {y} played:", y_stints)
print(y)
shoots_types = ["3pt", "2pt"]
df = pbp_df.query('stint1 in @x and player == "Y. Wetzell" and actionType in ["2pt", "3pt", "freethrow"] and success == 1')
# df.query('actionType == "2pt"')

df.shape


Stints where H. Besson played: {1, 3, 4, 5, 6, 7, 8, 9, 15, 16}
Stints where W. McDowell-White played: {1, 6, 7, 8, 9, 11, 12, 13, 14}
W. McDowell-White


(0, 20)

In [68]:
players = players_1

onoff_df = pd.DataFrame()  # Correlation matrix
for x in players:
    x_stints = set(players_stints[x])
    x_pbp_df = pbp_df.query('player in @x and actionType in ["2pt", "3pt", "freethrow"] and success == 1')
    c_2pt = x_pbp_df.query('actionType == "2pt"').shape[0]
    c_3pt = x_pbp_df.query('actionType == "3pt"').shape[0]
    c_1pt = x_pbp_df.query('actionType == "freethrow"').shape[0]
    x_pts = 2*c_2pt + 3*c_3pt + c_1pt
    for y in players:
        if x == y:
            v = -1
        else:
            y_stints = set(players_stints[y])
            together = x_stints.intersection(y_stints)
            alone = x_stints.difference(y_stints)
            # print(x, y, together, alone)

            together_df = x_pbp_df.query('stint1 in @together')
            c_2pt = together_df.query('actionType == "2pt"').shape[0]
            c_3pt = together_df.query('actionType == "3pt"').shape[0]
            c_1pt = together_df.query('actionType == "freethrow"').shape[0]
            together_pts = 2*c_2pt + 3*c_3pt + c_1pt

            alone_df = x_pbp_df.query('stint1 in @alone')
            c_2pt = alone_df.query('actionType == "2pt"').shape[0]
            c_3pt = alone_df.query('actionType == "3pt"').shape[0]
            c_1pt = alone_df.query('actionType == "freethrow"').shape[0]
            alone_pts = 2*c_2pt + 3*c_3pt + c_1pt

            print(x, y, x_pts, together_pts, alone_pts)



Y. Wetzell F. Delany 7 7 0
Y. Wetzell H. Besson 7 7 0
Y. Wetzell O. Dieng 7 2 5
Y. Wetzell W. McDowell-White 7 7 0
Y. Wetzell P. Siva 7 3 4
Y. Wetzell J. Martin 7 2 5
Y. Wetzell S. Timmins 7 0 7
Y. Wetzell K. Galloway 7 0 7
Y. Wetzell R. Loe 7 0 7
F. Delany Y. Wetzell 8 8 0
F. Delany H. Besson 8 5 3
F. Delany O. Dieng 8 6 2
F. Delany W. McDowell-White 8 2 6
F. Delany P. Siva 8 8 0
F. Delany J. Martin 8 6 2
F. Delany S. Timmins 8 0 8
F. Delany K. Galloway 8 0 8
F. Delany R. Loe 8 0 8
H. Besson Y. Wetzell 17 13 2
H. Besson F. Delany 17 13 2
H. Besson O. Dieng 17 10 5
H. Besson W. McDowell-White 17 6 9
H. Besson P. Siva 17 10 5
H. Besson J. Martin 17 11 4
H. Besson S. Timmins 17 2 13
H. Besson K. Galloway 17 2 13
H. Besson R. Loe 17 0 15
O. Dieng Y. Wetzell 3 0 3
O. Dieng F. Delany 3 3 0
O. Dieng H. Besson 3 0 3
O. Dieng W. McDowell-White 3 0 3
O. Dieng P. Siva 3 3 0
O. Dieng J. Martin 3 3 0
O. Dieng S. Timmins 3 3 0
O. Dieng K. Galloway 3 0 3
O. Dieng R. Loe 3 0 3
W. McDowell-White Y. We

In [108]:



# stints1_df.iloc[3]['lineup']

[2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16]